# Data Creation

## Conference and Edition


```bash
CREATE (ce:ConferenceEdition {name: 'Fifth Workshop on Computer Architecture for Non-Numeric Processing', year: 1980,month:'April',  venue: 'Pacific Grove', editionID:"conf/sigmod/80nnp"})

CREATE (c:Conference {  name: 'Computer Architecture'})


```` bash
CREATE CONSTRAINT ON (ce:ConferenceEdition)
ASSERT ce.editionID IS UNIQUE;

```bash
match (ce:ConferenceEdition {editionID:"conf/sigmod/80nnp"})
match (c:Conference {name:"Computer Architecture"})
merge (ce)-[:part_of] ->(c)

``` BASH
MATCH (ce:ConferenceEdition {editionID:"conf/sigmod/80nnp"})
MATCH (p:proceedings {key:"conf/sigmod/80nnp"})
MERGE (p)-[:of_edition] ->(ce)

<div style="background-color: white; padding: 10px; display: inline-block;">
  <img src="graph3.png" alt="Graph" />
</div>

Data creation for the queries to yield good results

```bash
// Create HAS_CITATION edges for three papers
UNWIND [
  // HanY10b is cited in three others
  {source:"journals/jicce/HanY10b",    target:"journals/jicce/Lim11b"},
  {source:"journals/jicce/HanY10b",    target:"journals/jicce/Kim09g"},
  {source:"journals/jicce/HanY10b",    target:"journals/jicce/PhamMLY08"},

  // Lim11b is cited in two others
  {source:"journals/jicce/Lim11b",    target:"journals/jicce/Oh12"},
  {source:"journals/jicce/Lim11b",    target:"journals/jicce/Kim09g"},

  // KimYS04 is cited in one other
  {source:"journals/jicce/KimYS04",   target:"journals/jicce/LeeKLK10"}
] AS row
// 1) make sure both nodes exist (stub if needed)
MERGE (src:article {key: row.source})
MERGE (tgt:article {key: row.target})
// 2) create the citation edge
MERGE (src)-[:HAS_CITATION]->(tgt);

```bash
UNWIND [
  {articleKey:"journals/jicce/HanY10b"},
  {articleKey:"journals/jicce/Lim11b"},
  {articleKey:"journals/jicce/KimYS04"}
] AS row

// 1) Find the article, proceedings, edition and conference
MATCH (a:article {key: row.articleKey})
MATCH (pr:proceedings {key: "conf/sigmod/80nnp"})
MATCH (ed:ConferenceEdition {editionID: "conf/sigmod/80nnp"})
MATCH (c:Conference {name: "Computer Architecture"})

// 2) Create or find a single booktitle for this edition
MERGE (bt:booktitle {
  title: "SIGMOD 1980 Proceedings"    // label it however makes sense
})

// 3) Link everything up
MERGE (a)-[:PRESENTED_AT]->(bt)
MERGE (pr)-[:PRESENTED_AT]->(bt)
MERGE (bt)-[:OF_EDITION]->(ed)
MERGE (ed)-[:PART_OF]->(c);
